In [1]:
from matplotlib.colors import ListedColormap # for grgphing decision boundaries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
data_folder = 'data'
X_train = pd.read_csv(f'./{data_folder}/X_train.csv')
y_train = pd.read_csv(f'./{data_folder}/y_train.csv')
X_test = pd.read_csv(f'./{data_folder}/X_test.csv')
y_test = pd.read_csv(f'./{data_folder}/y_test.csv')

In [3]:
class Network:
    def __init__(self):
        # layers/activations
        self.input = np.random.rand(4, 1)
        self.h1 = np.random.rand(2, 1)
        self.h2 = np.random.rand(3, 1)
        
        # weights 
        self.w_1 = np.random.rand(2, 4)
        self.w_2 = np.random.rand(3, 2)
        
        # biases 
        self.b_1 = np.random.rand(2, 1)
        self.b_2 = np.random.rand(3, 1)

        
    def one_hot(self,y):
        one_hot_y = np.zeros((y.size, y.max()+1))
        one_hot_y[np.range(y,size), y] = 1
        return one_hot_y.T

    def relu(self,activations):
        return np.maximum(0, activations)

    def relu_deriv(self, activations):
        return activations > 0

    def softmax(self, activations):
        return np.exp(activations) / np.sum(np.exp(activations))
    
    def feed_forward(self, X):
        # input
        # reshapre the input into vector form.
        # Example [1, 0, 0] -> [[1], [1], [1]]
        self.input = np.reshape(X, (-1,1))
        
        # input -> h1
        # the activations in the first hidden layer are given by the dot product 
        # of the weights by the input plus some biass its all then passed into
        # our activation function. relu(W_1*x+b_1)
        h1_activations = self.relu(np.dot(self.w_1, self.input) + self.b_1)
        self.h1 = h1_activations

        # h1 -> h2
        # the activations in the seocnd hidden layer (h_2) are given by the dot product 
        # of the second weights (w_2) by the previous activations (h1) plus the bias(b_2).
        # W_2*h1+b_2
        h2_activations = (np.dot(self.w_2, h1_activations) + self.b_2)
        self.h2 = h2_activations 

        # h2 -> output 
        # our output activtions/predictions are given by the second layer activations (h_2)
        # put into the softmax function. 
        output = self.softmax(self.h2)
        
        return (output, max(output))
    
    def back_prop(self, output, y, learning_rate=0.01):
        y = np.reshape(y, (-1,1))
#         print(input_)
        
        # Derivative of the cost function.
        # The cost function is C = (output - y)^2
        # The derivative of that is 2 (output - y)
        dc = (2 * (output - y))
        
        # Derivative of the cost function with respect to the weights (w_2).
        # The because our output is given by: output = w_2*h_1+b_2. The deriative
        # of the cost function with respect to w_2 is: dc * h1
        dc_dw2 = -learning_rate * dc.dot(self.h1.T)
        b_2 = -learning_rate * dc


        # Derivative of the cost function with respect to the first weights (w_1)
        # h1 is given by w_1*x+b_1
        h1_error = self.w_2.T.dot(dc) * self.relu_deriv(self.h1)
        dc_dw1 = -learning_rate * (h1_error.dot(self.input.T))
        b_1 = -learning_rate * h1_error
        
        self.w_1 +=  dc_dw1
        self.w_2 +=  dc_dw2
        
        self.b_1 += b_1
        self.b_2 += b_2
        return 0
       
    def get_accuracy(self, data):
        total = 0 
        for index, row in data.iterrows():
            # select the label
            y = row.tolist()[4:]
            # select the x
            X = row.tolist()[:4] 
            output, predicted = self.feed_forward(X)
            label_index = y.index(max(y))
            predicted_index = np.where(output==predicted)[0][0]
            if label_index == predicted_index:
                total += 1
        return total/data.shape[0]
        

    def train(self, X, Y, itterations):        
        y_dummies = pd.get_dummies(Y)
        data = pd.concat([X, y_dummies], axis=1)
        for i in range(itterations):
            for index, row in data.iterrows():
                # select the label
                y = row.tolist()[4:]
                # select the x
                x = row.tolist()[:4] 
                output_activations, prediction = self.feed_forward(x)
                self.back_prop(output_activations, y)

In [4]:
data = pd.concat([X_test, pd.get_dummies(y_test)], axis=1)

In [5]:
new_class = Network()

In [6]:
new_class.train(X_train, y_train, 100)

In [7]:
x = data.iloc[0].to_list()[:4]
x

[6.2, 2.8, 4.8, 1.8]

In [8]:
y = data.iloc[0].to_list()[4:]
y

[0.0, 0.0, 1.0]

In [9]:
new_class.feed_forward(x)

(array([[1.32233102e-10],
        [6.34936913e-02],
        [9.36506309e-01]]),
 array([0.93650631]))

In [10]:
new_class.get_accuracy(data)

0.9210526315789473